In [1]:
%load_ext autoreload
%autoreload 2

In [80]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
from tqdm import tqdm
import torch

from qasper.dataset_reader import QasperReader
from qasper.models import qasper, gpt35

In [3]:
dataset = load_dataset("allenai/qasper")

In [81]:
reader = QasperReader()

In [82]:
def instance_generator(split):
    for article in split:
        for instance in reader._article_to_instances(article):
            yield instance

In [83]:
instances = instance_generator(dataset['train'])

In [84]:
instance = next(instances)

In [85]:
instance.keys()

dict_keys(['question_with_context', 's_question_with_context', 'paragraph_indices', 'global_attention_mask', 'evidence', 'answer', 'metadata'])

In [86]:
question_id = instance['metadata']['question_id']
print(question_id)

753990d0b621d390ed58f20c4d9e4f065f0dc672


In [88]:
print(instance['s_question_with_context'])

What is the seed lexicon? Introduction Affective events BIBREF0 are events that typically affect people in positive or negative ways. For example, getting money and playing sports are usually positive to the experiencers; catching cold and losing one's wallet are negative. Understanding affective events is important to various natural language processing (NLP) applications such as dialogue systems BIBREF1, question-answering systems BIBREF2, and humor recognition BIBREF3. In this paper, we work on recognizing the polarity of an affective event that is represented by a score ranging from $-1$ (negative) to 1 (positive). Learning affective events is challenging because, as the examples above suggest, the polarity of an event is not necessarily predictable from its constituent words. Combined with the unbounded combinatorial nature of language, the non-compositionality of affective polarity entails the need for large amounts of world knowledge, which can hardly be learned from small annot

## Model

In [96]:
answers = qasper.predict(instance)

In [97]:
answers

['A recurrent pattern with the pattern of BIBREF and a linear sequence as the expression type']

In [105]:
gpt35_answer = gpt35.predict(instance)

In [99]:
# answers = []
# answer_token_log_probs = []
# normalized_answer_log_probs = []

# # https://huggingface.co/docs/transformers/en/internal/generation_utils
# output_sequences = generation_output.sequences.tolist()
# output_scores = generation_output.scores

# for answer_id, sequence in enumerate(output_sequences):
#     # answers
#     answers.append(tokenizer.decode(sequence, skip_special_tokens=True))
    
#     # answer_token_log_probs
#     answer_token_log_probs.append([])
#     word_pieces = tokenizer.convert_ids_to_tokens(sequence) # what does this do?
#     # Skipping the first token id because that is the sep token, and the scores start from the
#     # second token.
#     for token_id, token, token_scores in zip(sequence[1:], word_pieces[1:], output_scores):
#         if token == tokenizer.eos_token:
#             break
        
#         token_log_prob = torch.log(torch.softmax(token_scores[answer_id], dim=0)[token_id]).tolist()
        
#         answer_token_log_probs[-1].append(token_log_prob)
    
#     normalized_answer_log_probs.append(sum(answer_token_log_probs[-1]) / len(answer_token_log_probs[-1]))


In [100]:
# output_data = {
#     "question_id": question_id,
#     "answers": answers,
#     "answer_token_log_probs": answer_token_log_probs,
#     "normalized_answer_log_probs": normalized_answer_log_probs
# }

In [101]:
# output_data

## Evaluate F1 score

In [102]:
from qasper import evaluator

In [103]:
# evaluator.evaluate(output_data, instance)

In [104]:
evaluator.token_f1_score(answers[0], instance['answer'])

0.16

In [106]:
evaluator.token_f1_score(gpt35_answer, instance['answer'])

0.08743169398907102